In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# models
from sklearn.linear_model import LogisticRegression
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks


# model evaluation
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split

# plotting
import seaborn as sns
from matplotlib.pyplot import scatter
from matplotlib import rcParams
import matplotlib.pyplot as plt
size = 20
# square plots
rcParams['figure.figsize'] = size,size


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv", index_col = 'id')


check for missing values

In [ ]:
# sum(df.isna().sum())

don't need id for training and validation. Can match on rows for test set.

In [ ]:
# id = df.pop('id')

realized later I should just make id the index

In [ ]:
# pd.set_option('display.max_columns', None)
# df.describe()

In [ ]:
# mat = df.corr()

In [ ]:
# mat.style.background_gradient()

the above method didn't do much more than print the correlation matrix. Need to tune the params?

In [ ]:
# sns.heatmap(mat)

Seems to have two predictors with a strong (linear) relationship with the target. Around f21 and f178. Let's look at those regions again and figure out exactly which variables are strongly related.

In [ ]:
# mat.iloc[-1,19:26]

In [ ]:
# mat.iloc[-1, 176:182]

f179 and f22 have the highest (linear) correlation with the target

In [ ]:
# hand_picked_subset = df.loc[:, ["f22", "f179"]]

In [ ]:
# hand_picked_subset.describe()

f22 is a boolean variable while f179 is a continuous variable.

In [ ]:
# y = df.loc[:,"target"]
# y.describe()

target is a boolean value also. 

plot the independent variables

In [ ]:
# scatter(df.loc[:, "f22"], df.loc[:, "f179"])

In [ ]:
# scatter(df.loc[:, "f22"], df.loc[:, "f179"])

In [ ]:
# scatter( df.loc[:, "f179"], df.loc[:, "target"])

In [ ]:
# scatter(df.loc[:, "f22"], df.loc[:, "target"])

In [ ]:
# logit = LogisticRegression()
# logit.fit(hand_picked_subset,  df.loc[:, "target"])

y axis = probability target value is 1 given the 

In [ ]:
# scatter(x =hand_picked_subset.iloc[:, 0], y =logit.predict_proba(hand_picked_subset)[:,1])

We see two distict clusters for f179. this seems to make sense given our model (f22 is a boolean). Let's check it's performance predicting the test.csv data (which is confusingly our validation set!)

In [ ]:
# scatter(x =hand_picked_subset.iloc[:, 1], y =logit.predict_proba(hand_picked_subset)[:,1])

plots of the pairwise regressions

In [ ]:
# df_plt = df.loc[:, ["f22","f179","target"]]
# sns.lmplot(x= "f22", y = "target", logistic=True, data = df)

In [ ]:
# df_plt = df.loc[:, ["f22","f179","target"]]
# sns.lmplot(x= "f172", y = "target", logistic=True, hue="f22", data = df_plt)

In [ ]:
# dft = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv", index_col="id")

In [ ]:
# ty = dft.loc[:, "target"]
# tx = dft.loc[:, ["f22", "f179"]]

In [ ]:
# y_hat = logit.predict_proba(tx)

In [ ]:
# len(y_hat)

In [ ]:


# false_pos_rate = []
# true_pos_rate  = []
# for i in range(2): # binary classification
#     fpr, tpr, t = roc_curve(y_true = ty, y_score = y_hat[:, i])
#     false_pos_rate.append(fpr)
#     true_pos_rate.append(tpr)
#     area = auc(fpr, tpr)
#     print(area)
#     plt.figure()
#     plt.plot(tpr, fpr, label='ROC curve -- area under curve = {area:0.3f}'.format(area = area), lw = 4)
#     plt.plot([0, 1], [0, 1], linestyle ='--', lw =4)
#     plt.xlim([0, 1])
#     plt.ylim([0, 1])
#     plt.legend(loc = "upper left")
#     plt.show()
#     print()


In [ ]:
# logit2 = LogisticRegression()
# nrows, ncols = df.shape
# logit2.fit(df.iloc[:, 0:ncols-1], df.loc[:, "target"])


In [ ]:
nrows, ncols = df.shape
# ty = dft.loc[:, "target"]
# txf = dft.loc[:, df.columns != "target"]

In [ ]:
# ty_logit = logit2.predict_proba(txf)
# t, f, _ = roc_curve(y_true = ty, y_score=ty_logit[:, 1])
# print(auc(t,f))

In [ ]:
# def plot_roc_curves(true_y, pred_y):
#     for i in range(2): # binary classification
#         fpr, tpr, t = roc_curve(y_true =  true_y, y_score = pred_y[:, i])
#         area = auc(fpr, tpr)
#         plt.figure()
#         plt.plot(tpr, fpr, label='ROC curve -- area under curve = {area:0.3f}'.format(area = area), lw = 4)
#         plt.plot([0, 1], [0, 1], linestyle ='--', lw =4)
#         plt.xlim([0, 1])
#         plt.ylim([0, 1])
#         plt.legend(loc = "upper left")
#         plt.show()
        

In [ ]:
# plot_roc_curves(ty, ty_logit)

using the full dataset is essentially the same as flipping a coin!!!! 

In [ ]:
nrows, ncols = df.shape
model = keras.Sequential([
    layers.Dense(256, activation='elu', input_shape = [ncols-1]),
    layers.Dense(64, activation='elu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile( optimizer = "adam",
              loss = keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy']
             )

# not best suited for binary classification
# loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
# CategoricalCrossentropy too but it'd require one hot encoded y values


In [7]:
X_train, X_val, y_train, y_val = train_test_split(df.loc[:, df.columns != "target"],  df.loc[:, "target"], train_size=0.75)

In [8]:
X_val.shape,X_train.shape

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 5, min_delta=0.001, restore_best_weights = True )
fit_hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=10000, epochs = 50, callbacks = [early_stopping] )
h = pd.DataFrame(fit_hist.history)
h.plot()

that's strange. It seems to have learned all it could after one epoch. Let's write out the result for our first submission.  

first clean up some memory. 

In [ ]:
# del df

In [ ]:
del X_train
del X_val
del y_train
del y_val
del hand_picked_subset
del y
del tx
del ty
del y_hat


# del 

In [ ]:
# y_hat = model.predict(dft.loc[:, df.columns != "target"])
y_hat = model.predict(dft)

realized I imported the wrong file. test.csv has 500k rows, train has 1m. 

This model is overfit. Also, was using the wrong loss function. 

In [ ]:
# for i in range(0, dft.shape[1], step = 100 ):
#     batch = model.predict(dft.iloc[i:i+100, :])
#     batch.to_csv("../input/tabular-playground-series-oct-2021/submission.csv", mode='a')
    

In [ ]:
pd.DataFrame(y_hat, index=dft.index).to_csv("/kaggle/working/submission.csv")

In [ ]:
model = keras.Sequential([
    layers.Dense(1024, activation='swish', input_shape = [ncols-1]),
    layers.Dense(512, activation='relu'),
    layers.Dropout(rate=0.25),
    layers.Dense(256, activation='relu'),
    layers.Dense(64, activation="elu"),
    layers.Dense(1, activation='sigmoid')
])

model.compile( optimizer = "adam",
              loss = keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy']
             )
early_stopping = callbacks.EarlyStopping(patience = 20, min_delta=0.001, restore_best_weights = True )
fit_hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=500, epochs = 50, callbacks = [early_stopping] )

h = pd.DataFrame(fit_hist.history)
h.plot()



In [10]:
model = keras.Sequential([
    layers.Conv1D(128, 3,  input_shape = [ncols-1], activation = "gelu"),
    layers.MaxPool1D(pool_size = 3, strides = 2),
    layers.Dense(512, activation='swish',),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=0.25),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation="elu"),
    layers.Dense(1, activation='sigmoid')
])

model.compile( optimizer = "adam",
              loss = keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy']
             )
early_stopping = callbacks.EarlyStopping(patience = 5, min_delta=0.001, restore_best_weights = True )
fit_hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=500, epochs = 50, callbacks = [early_stopping] )

h = pd.DataFrame(fit_hist.history)
h.plot()

